In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [112]:
hc = 12.39841875 #keV*Angström
d_si = 3.1358

In [93]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen/Messdaten/'

In [3]:
var_pc = pc_frederik

In [5]:
cd $var_pc

/home/frederik/Dokumente/FP/Roentgen/Messdaten


# Implementirung der Funktionen für die Auswertung

## Die Peak sollen mit der Gaussverteilung angefitten werde, diese wird hier implementier, wie ein wrappen für automatische fits

# Gauss
<h1>$f(x) = \frac{a}{\sqrt{2 \cdot pi \cdot \sigma^2}} \cdot e^{- \frac{{(x- \sigma )}^2}{2 \cdot \sigma^2}} $ </h1>

In [6]:
def gauss(x,a,mu,sigma):
    return (a/(np.sqrt(2*3.14*sigma**2))*np.exp(-((x-mu)**2)/(2.*sigma**2)))

# ${\chi_{red}}^2$ / Goodness of a fit
<h1> ${\chi_{red}}^2 = \frac{1}{\nu}\sum\frac{{(O-E)}^2}{\sigma^2}$ 

In [7]:
def red_chi_gauss(data, model, deg):
    chisq = np.sum(np.square(data-model)/np.power(data,0.5))
    nu = data.size-1-deg    # -4: -1 for the equation and -3 from the degrees of freedom
    if nu is 0:
        return "err: nu = 0"
    return chisq/nu

In [8]:
def fit_gauss(x,data,guess=[1,1,1]):
    '''
        returns the parameter from the fit, the standard deviation and the reduced chisquare
    '''
    para, cov = opt.curve_fit(gauss,x,np.sqrt(data),guess,maxfev=10000)
    red_chi = red_chi_gauss(data,gauss(x,para[0],para[1],para[2]),3)
    return para, np.sqrt(np.diag(cov)), red_chi

# Bragg-reflexion
$n \cdot \lambda = 2 \cdot d \cdot sin(\theta)$

# Energie
$E = \frac{h \cdot c}{2 \cdot d \cdot sin(\theta)}$

In [9]:
def energie(phi,delta_phi):
    theta = phi/2
    e = hc/(2*d_si*np.sin(theta*(3.14/180)))
    delta_e = (e*delta_phi/2)/np.tan(theta*(3.14/180))
    return e, delta_e

In [10]:
def intensität(a,delta_a,sigma,delta_sigma):
    i = a/(np.sqrt(2*3.14*sigma**2))
    delta_i = np.sqrt((delta_a/np.sqrt(2*3.14*sigma**2))**2 + ((a*delta_sigma)/(np.sqrt(2*3.14*sigma**4)))**2)
    return i, delta_i

In [11]:
def d(e,delta_e,phi,delta_phi,n):
    d = (n*hc)/(2*e*np.sin((phi/2)*(3.14/180)))
    delta_d = d*np.sqrt(((delta_phi/2)/np.tan((phi/2)*(3.14/180)))**2+(delta_e/e)**2)

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = ??, A$_1$ = ??
+ U$_2$ = ??, A$_2$ = ??
+ U$_3$ = ??, A$_3$ = ??

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdefraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

# Ersterwert: Spannung
# Zweiterwert: Anodenstrom


# Vorbereitung für den Voigt-Fit

In [73]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
mulit_voigt_1 = voigt_1 + voigt_2

In [95]:
data = np.loadtxt("Si111_40_30.xy").T
plt.plot(data[0],data[1])
plt.show()

In [78]:
params_1 = voigt_1.guess(data[1],x=data[0])
params_2 = voigt_2.guess(data[1],x=data[0])
params_3 = voigt_3.guess(data[1],x=data[0])
params_mulit = mulit_voigt_1.make_params()

params_mulit['voigt_1center'].set(28.43)
params_mulit['voigt_1center'].set(28.50)
params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [90]:
np.arange(np.where(data[0] > 28.2)[0][0], np.where(data[0] < 28.7)[0][0])

array([], dtype=int64)

In [92]:
np.where(data[0] < 28.7)

(array([   0,    1,    2, ..., 3276, 3277, 3278]),)

In [80]:
fit_1 = voigt_1.fit(data[1],params_1,x=data[0])
fit_2 = voigt_2.fit(data[1],params_2,x=data[0])
fit_3 = voigt_3.fit(data[1],params_3,x=data[0])
fit_mulit = mulit_voigt_1.fit(data[1],params_mulit,x=data[0])
print(fit_mulit.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_1') + Model(voigt, prefix='voigt_2'))
[[Fit Statistics]]
    # function evals   = 7
    # data points      = 27526
    # variables        = 6
    chi-square         = nan
    reduced chi-square = nan
[[Variables]]
    voigt_2center:     -inf        +/- 0        (-0.00%) (init=-inf)
    voigt_1amplitude:  -inf        +/- 0        (-0.00%) (init=-inf)
    voigt_1fwhm:        0          +/- 0          == '3.6013100*voigt_1sigma'
    voigt_2gamma:       0          +/- 0          == 'voigt_2sigma'
    voigt_2fwhm:        0          +/- 0          == '3.6013100*voigt_2sigma'
    voigt_1center:      28.5000000 +/- 0        (0.00%) (init= 28.5)
    voigt_1sigma:       0          +/- 0         (init= 0)
    voigt_1gamma:       0          +/- 0          == 'voigt_1sigma'
    voigt_2amplitude:  -inf        +/- 0        (-0.00%) (init=-inf)
    voigt_2sigma:       0          +/- 0         (init= 0)
[[Correlations]] (unreported correlations are <  0.100)


In [63]:
peak_1_range = np.arange(np.where( 25.5 < data[0])[0][0],np.where( data[0] > 25.8)[0][0])
peak_2_range = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
peak_3_range = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])

In [64]:
params_1 = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])
params_2 = voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range])
params_3 = voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [70]:
fit_1 = voigt_1.fit(data[1][peak_1_range],params_1,x=data[0][peak_1_range],weights=1/np.sqrt(data[1][peak_1_range]))
fit_2 = voigt_2.fit(data[1][peak_2_range],params_2,x=data[0][peak_2_range],weights=1/np.sqrt(data[1][peak_2_range]))
fit_3 = voigt_3.fit(data[1][peak_3_range],params_3,x=data[0][peak_3_range],weights=1/np.sqrt(data[1][peak_3_range]))

In [84]:
fit_1.plot()
fit_2.plot()
fit_3.plot()
plt.show()

In [72]:
print(fit_1.fit_report())
print(fit_2.fit_report())
print(fit_3.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 259
    # data points      = 71
    # variables        = 4
    chi-square         = 3603.669
    reduced chi-square = 53.786
[[Variables]]
    voigt_1center:      25.6459482 +/- 0.000345 (0.00%) (init= 25.64617)
    voigt_1sigma:       0.01038146 +/- 0.000832 (8.02%) (init= 0.01357791)
    voigt_1gamma:       0.01311903 +/- 0.000584 (4.45%) (init= 1)
    voigt_1amplitude:   1179.61910 +/- 17.04245 (1.44%) (init= 1214.115)
    voigt_1fwhm:        0.03738687 +/- 0.002997 (8.02%)  == '3.6013100*voigt_1sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.805 
    C(voigt_1gamma, voigt_1amplitude)  =  0.181 
    C(voigt_1sigma, voigt_1amplitude)  = -0.144 
[[Model]]
    Model(voigt, prefix='voigt_2')
[[Fit Statistics]]
    # function evals   = 239
    # data points      = 22
    # variables        = 4
    chi-square         = 4583.869
    reduced chi-squ

In [96]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.plot(data[0][peak_1_range],fit_1.best_fit)
plt.plot(data[0][peak_2_range],fit_2.best_fit)
plt.plot(data[0][peak_3_range],fit_3.best_fit)
plt.show()

ValueError: x and y must have same first dimension

In [26]:
multivoigt = voigt_1 + voigt_2 + voigt_3

In [27]:
params = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])

In [28]:
params.update(voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range]))

In [29]:
params.update(voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range]))

In [38]:
err = np.sqrt(data[1])
err = np.where(err==0, 1, err)

In [42]:
fit_4 = multivoigt.fit(data[1],params,x=data[0],weights=1/err)

In [43]:
fit_4.best_values

{'voigt_1amplitude': 1219.38946039998,
 'voigt_1center': 25.646027872265016,
 'voigt_1gamma': 0.012638462997359046,
 'voigt_1sigma': 0.012638462997359046,
 'voigt_2amplitude': 2734.0711559760721,
 'voigt_2center': 28.434221054683281,
 'voigt_2gamma': 0.016102623523348125,
 'voigt_2sigma': 0.016102623523348125,
 'voigt_3amplitude': 1575.2818528179009,
 'voigt_3center': 28.508759553087831,
 'voigt_3gamma': 0.011411860899229964,
 'voigt_3sigma': 0.011411860899229964}

In [44]:
print(fit_4.fit_report())

[[Model]]
    ((Model(voigt, prefix='voigt_1') + Model(voigt, prefix='voigt_2')) + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 51
    # data points      = 27526
    # variables        = 9
    chi-square         = 901715.374
    reduced chi-square = 32.769
[[Variables]]
    voigt_1center:      25.6460278 +/- 0.000269 (0.00%) (init= 25.64617)
    voigt_1sigma:       0.01263846 +/- 0.000159 (1.26%) (init= 0.01357791)
    voigt_1gamma:       0.01263846 +/- 0.000159 (1.26%)  == 'voigt_1sigma'
    voigt_1amplitude:   1219.38946 +/- 12.93579 (1.06%) (init= 1214.115)
    voigt_1fwhm:        0.04551502 +/- 0.000576 (1.26%)  == '3.6013100*voigt_1sigma'
    voigt_2fwhm:        0.05799053 +/- 0.000689 (1.19%)  == '3.6013100*voigt_2sigma'
    voigt_2center:      28.4342210 +/- 0.000308 (0.00%) (init= 28.43486)
    voigt_2amplitude:   2734.07115 +/- 26.99991 (0.99%) (init= 1946.006)
    voigt_2sigma:       0.01610262 +/- 0.000191 (1.19%) (init= 0.0149357)
    voigt_2g

In [ ]:
fit_4.plot()
plt.legend(loc='best')
plt.show()

In [97]:
plt.plot(data[0],fit_4.best_fit,'r',label='Multi-Voigt-Fit')
plt.errorbar(data[0],data[1],np.sqrt(data[1]),label='Messdaten')
plt.xlim([25,29])
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.show()